In [1]:
import numpy as np
import fluidfoam as ff
import matplotlib.pyplot as plt
from scipy import interpolate
import joblib
from tqdm.notebook import tqdm
import time

In [2]:
rootPath = '/home/whw/OpenFOAM/whw-v2412/run/coupledCases/'

Gr_all = [1e4, 2e4, 3e4, 4e4]
Ha_all = [1e2, 2e2, 3e2, 4e2, 5e2]

Gr_all_str = [f"{x:.0e}".replace("+0", "").replace("+", "") for x in Gr_all]
Ha_all_str = [f"{x:.0e}".replace("+0", "").replace("+", "") for x in Ha_all]

In [3]:
test_path = rootPath + 'Gr1e4,Ha1e2'
x,y,z = ff.readof.readmesh(test_path)
points_original = np.vstack((x, y, z)).T

Reading file /home/whw/OpenFOAM/whw-v2412/run/coupledCases/Gr1e4,Ha1e2/constant/polyMesh/owner
Reading file /home/whw/OpenFOAM/whw-v2412/run/coupledCases/Gr1e4,Ha1e2/constant/polyMesh/faces
Reading file /home/whw/OpenFOAM/whw-v2412/run/coupledCases/Gr1e4,Ha1e2/constant/polyMesh/points
Reading file /home/whw/OpenFOAM/whw-v2412/run/coupledCases/Gr1e4,Ha1e2/constant/polyMesh/neighbour


In [4]:
def geometric_sum(a1, r, n):
    """
    计算等比数列的和

    参数:
    a1 -- 首项
    r -- 公比
    n -- 项数

    返回:
    等比数列的和
    """
    # 使用公式计算等比数列的和
    if r == 1:
        return a1 * n  # 当公比为1时，求和就是首项乘以项数
    else:
        return a1 * (1 - r**n) / (1 - r)

In [5]:
x_inter=[]
y_inter=[]
for i in range(16):
    x_inter.append(geometric_sum(0.0008453981596,1.05,i))
    y_inter.append(geometric_sum(0.0008453981596,1.05,i))

for i in range(16):
    x_inter.append(0.02+geometric_sum(0.001757522057,0.9523809523809523,i))
    y_inter.append(0.02+geometric_sum(0.001757522057,0.9523809523809523,i))

z_inter = np.linspace(0, 0.3, 16)

X, Y, Z = np.meshgrid(x_inter, y_inter, z_inter)

points_inter = np.vstack((X.ravel(), Y.ravel(), Z.ravel())).T

In [6]:
label = []
w_all = []
T_all = []

for i,Gr in tqdm(enumerate(Gr_all), desc="Outer loop", position=0, total=len(Gr_all)):
    for j,Ha in tqdm(enumerate(Ha_all), desc="Inner loop", position=1, total=len(Ha_all), leave=False):
        label.append('Gr' + str(Gr) + ',Ha' + str(Ha))
        current_path = rootPath + 'Gr' + Gr_all_str[i] + ',Ha' + Ha_all_str[j]
        w_original = ff.readof.readfield(current_path,'1000','U')[2]
        T_original = ff.readof.readfield(current_path,'1000','T', verbose=False)

        Interp_w = interpolate.NearestNDInterpolator(points_original, w_original)
        Interp_T = interpolate.NearestNDInterpolator(points_original, T_original)

        w_inter = Interp_w(points_inter)
        T_inter = Interp_T(points_inter)

        w_inter = 1e5 * w_inter
        T_inter = 1e3 * (T_inter - 723.15)

        w_all.append(w_inter.reshape(X.shape))
        T_all.append(T_inter.reshape(X.shape))

        

Outer loop:   0%|          | 0/4 [00:00<?, ?it/s]

Inner loop:   0%|          | 0/5 [00:00<?, ?it/s]

Reading file /home/whw/OpenFOAM/whw-v2412/run/coupledCases/Gr1e4,Ha1e2/1000/U
Reading file /home/whw/OpenFOAM/whw-v2412/run/coupledCases/Gr1e4,Ha2e2/1000/U
Reading file /home/whw/OpenFOAM/whw-v2412/run/coupledCases/Gr1e4,Ha3e2/1000/U
Reading file /home/whw/OpenFOAM/whw-v2412/run/coupledCases/Gr1e4,Ha4e2/1000/U
Reading file /home/whw/OpenFOAM/whw-v2412/run/coupledCases/Gr1e4,Ha5e2/1000/U


Inner loop:   0%|          | 0/5 [00:00<?, ?it/s]

Reading file /home/whw/OpenFOAM/whw-v2412/run/coupledCases/Gr2e4,Ha1e2/1000/U
Reading file /home/whw/OpenFOAM/whw-v2412/run/coupledCases/Gr2e4,Ha2e2/1000/U
Reading file /home/whw/OpenFOAM/whw-v2412/run/coupledCases/Gr2e4,Ha3e2/1000/U
Reading file /home/whw/OpenFOAM/whw-v2412/run/coupledCases/Gr2e4,Ha4e2/1000/U
Reading file /home/whw/OpenFOAM/whw-v2412/run/coupledCases/Gr2e4,Ha5e2/1000/U


Inner loop:   0%|          | 0/5 [00:00<?, ?it/s]

Reading file /home/whw/OpenFOAM/whw-v2412/run/coupledCases/Gr3e4,Ha1e2/1000/U
Reading file /home/whw/OpenFOAM/whw-v2412/run/coupledCases/Gr3e4,Ha2e2/1000/U
Reading file /home/whw/OpenFOAM/whw-v2412/run/coupledCases/Gr3e4,Ha3e2/1000/U
Reading file /home/whw/OpenFOAM/whw-v2412/run/coupledCases/Gr3e4,Ha4e2/1000/U
Reading file /home/whw/OpenFOAM/whw-v2412/run/coupledCases/Gr3e4,Ha5e2/1000/U


Inner loop:   0%|          | 0/5 [00:00<?, ?it/s]

Reading file /home/whw/OpenFOAM/whw-v2412/run/coupledCases/Gr4e4,Ha1e2/1000/U
Reading file /home/whw/OpenFOAM/whw-v2412/run/coupledCases/Gr4e4,Ha2e2/1000/U
Reading file /home/whw/OpenFOAM/whw-v2412/run/coupledCases/Gr4e4,Ha3e2/1000/U
Reading file /home/whw/OpenFOAM/whw-v2412/run/coupledCases/Gr4e4,Ha4e2/1000/U
Reading file /home/whw/OpenFOAM/whw-v2412/run/coupledCases/Gr4e4,Ha5e2/1000/U


In [37]:
w_all[15][:,:,8].max()

5.8013336

In [8]:
w_all = np.array(w_all)

T_all = np.array(T_all)

In [9]:
input_all = []
for Gr in Gr_all:
    for Ha in Ha_all:
        input_all.append(np.array([Gr/1e4, Ha/1e2]))
input_all = np.array(input_all)

In [10]:

joblib.dump(input_all, 'preData/nonuni/input_all.joblib')
joblib.dump(w_all, 'preData/nonuni/w_train.joblib')
joblib.dump(T_all, 'preData/nonuni/T_train.joblib')
joblib.dump(label, 'preData/nonuni/label')

['preData/nonuni/label']